Idea code name: `S1P10`

In [1]:
from keras.layers import Input, Dense, LSTM, PReLU, Reshape, Dropout, Activation, BatchNormalization
from keras.models import model_from_json, Model
import keras

from s1p10_model.resnet8_body import resnet8_body

from s1p10_training_utils import save_lstm, DataGenerator

import json
import time

import numpy as np

Using TensorFlow backend.


## Load weight of each angle class

In [2]:
with open('./s1p10_data/s1p10_classes_weight.json', 'r') as fp:
    classes_weight = json.load(fp)

## Load bins edge

In [3]:
bins_edge = np.load('./s1p10_data/s1p10_bins_edge.npy')

# Global Config

In [4]:
IMAGE_SHAPE = (200, 200, 1)
NUM_CLASSES = len(classes_weight)
NUM_PREDICT = 5
LSTM_NUM_HIDDEN_STATE = 64
WEIGHT_REGU = 1e-3

# Model acrchitect

1 Image -> ResNet-8 -> LSTM_1 -> LSTM_2 (return state = True) (use this to sample prediction) -> Dense + Softmax 
-> 10 steering angle ID

## 1.2 Create Encoder
Encoder is body of ResNet-8 from `Drone-Net` since it's already trained to recognize road curve -> helpful spatial information


In [5]:
encoder = resnet8_body(IMAGE_SHAPE)
# load resnet8 weights
encoder.load_weights("./s1p10_model/named_resnet8_best_weights.h5", by_name=True)
for l in encoder.layers:
    l.trainable = False
encoder.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 200, 200, 1)  0                                            
__________________________________________________________________________________________________
conv_0 (Conv2D)                 (None, 100, 100, 32) 832         input_1[0][0]                    
__________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)  (None, 49, 49, 32)   0           conv_0[0][0]                     
__________________________________________________________________________________________________
bn_1_a (BatchNorma

## 1.3 Create Decoder
Decoder is comprised of 2 LSTM & 2 Dense layers, last Dense layer is activated by Softmax function

In [6]:
LSTM_cell = LSTM(LSTM_NUM_HIDDEN_STATE, 
                 return_state=True, 
                 kernel_regularizer=None,
                 recurrent_regularizer=None,
                 bias_regularizer=None,
                 dropout=0.0, 
                 recurrent_dropout=0.0)  # use to sample angle

In [7]:
def dense_classifier(input_shape, num_classes, weight_regu_param):
    """
    Define classifier made of several Dense layers
    """
    X_in = Input(shape=input_shape)
    
    # 1st layer
    X = BatchNormalization()(X_in)
    
    X = Dense(128)(X)
    
    X = Activation('relu')(X)
    
    # 2nd layer
    X = Dense(64)(X)
    
    X = Activation('relu')(X)
    
    # 3rd layer
    X = Dense(num_classes)(X)
    
    X_out = Activation('softmax')(X)
    
    model = Model(inputs=[X_in], outputs=[X_out], name='classifier_body')
    return model
    

In [8]:
classifier = dense_classifier((LSTM_NUM_HIDDEN_STATE, ), NUM_CLASSES, weight_regu_param=WEIGHT_REGU)

In [9]:
classifier.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 64)                0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 64)                256       
_________________________________________________________________
dense_1 (Dense)              (None, 128)               8320      
_________________________________________________________________
activation_8 (Activation)    (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 64)                8256      
_________________________________________________________________
activation_9 (Activation)    (None, 64)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 114)               7410      
__________

## 1.4 Create Model

In [10]:
def s1p10_model(input_shape, encoder, LSTM_cell, classifier, lstm_num_hidden_state, num_classes, num_labels):
    """
    Define see-1-predict-10 model
    
    Input:
        input_shape (tuple): shape of input image inputted to encoder
        encoder (keras.Model)
        LSTM_cell (keras.layers)
        classifier (keras.Model)
        
    Output:
        keras.Model
    """
    X_in = Input(shape=input_shape, name="image_in")
    
    # extract feature vector
    X_feature = encoder(X_in)
    X_feature = Reshape((1, -1))(X_feature)  # shape = (None, 1, 6272)
    
    # Initialize input to LSTM
    a0 = Input(shape=(LSTM_NUM_HIDDEN_STATE, ), name="a0")
    c0 = Input(shape=(LSTM_NUM_HIDDEN_STATE, ), name="c0")
    
    # True label of previous prediction. This will be concatenated with X_feature to make X
    y_init = [Input(shape=(num_classes, ), name="y_%d" % i) for i in range(num_labels)]  
    
    a = a0
    c = c0
    
    outputs = []
    
    for i in range(num_labels):
        # concatenate y & X_feature
        y = Reshape((1, -1))(y_init[i])
        X = keras.layers.concatenate([X_feature, y], axis=-1)
        
        # propagate X through LSTM_cell
        a, _, c = LSTM_cell(X, initial_state=[a, c])
        
        # propagate hidden state "a" through classifier to get steering angle
        y = classifier(a)
        
        # store y
        outputs.append(y)
    
    model = Model(inputs=[X_in, a0, c0] + y_init, outputs=outputs)
    return model
        

In [11]:
model = s1p10_model(IMAGE_SHAPE, 
                    encoder, 
                    LSTM_cell, 
                    classifier, 
                    LSTM_NUM_HIDDEN_STATE, 
                    NUM_CLASSES, 
                    NUM_PREDICT)

model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
image_in (InputLayer)           (None, 200, 200, 1)  0                                            
__________________________________________________________________________________________________
resnet8 (Model)                 (None, 6272)         309088      image_in[0][0]                   
__________________________________________________________________________________________________
y_0 (InputLayer)                (None, 114)          0                                            
__________________________________________________________________________________________________
reshape_1 (Reshape)             (None, 1, 6272)      0           resnet8[1][0]                    
__________________________________________________________________________________________________
reshape_2 

In [12]:
model.compile(optimizer='Adadelta', loss='categorical_crossentropy', metrics=['accuracy'])

# Training

In [13]:
batch_size = 100

gen_param = {'data_root_dir': "/home/user/Bureau/Dataset/udacity/", 
             'img_shape': IMAGE_SHAPE, 
             'num_class': NUM_CLASSES, 
             'num_prediction': NUM_PREDICT, 
             'bins_edge': bins_edge,
             'batch_size': batch_size, 
             'shuffle': True, 
             'lstm_dim_hidden_states': LSTM_NUM_HIDDEN_STATE,
             'flip_prob': 0.5}

train_gen = DataGenerator("./s1p10_data/s1p10_CH2_002_output_training.csv", **gen_param)
val_gen = DataGenerator("./s1p10_data/s1p10_CH2_002_output_validation.csv", **gen_param)

In [14]:
time_str = time.strftime("%Y_%m_%d_%H_%M")
log_dir = './s1p10_logs/' + time_str
tb_callback = keras.callbacks.TensorBoard(log_dir=log_dir,  
                                          batch_size=batch_size, 
                                          update_freq='epoch')

In [15]:
model.fit_generator(train_gen,
                    class_weight=classes_weight,
                    epochs=37,
                    validation_data=val_gen,
                    initial_epoch=7,
                    callbacks=[tb_callback])

Instructions for updating:
Use tf.cast instead.
Epoch 8/37
182/182 [==============================] - 144s 793ms/step - loss: 15.0557 - classifier_body_loss: 3.0048 - classifier_body_acc: 0.2199 - classifier_body_acc_1: 0.2201 - classifier_body_acc_2: 0.2177 - classifier_body_acc_3: 0.2164 - classifier_body_acc_4: 0.2199 - val_loss: 28.5934 - val_classifier_body_loss: 6.0274 - val_classifier_body_acc: 0.1767 - val_classifier_body_acc_1: 0.1736 - val_classifier_body_acc_2: 0.1720 - val_classifier_body_acc_3: 0.1720 - val_classifier_body_acc_4: 0.1720
Epoch 9/37
182/182 [==============================] - 142s 779ms/step - loss: 13.5607 - classifier_body_loss: 2.7059 - classifier_body_acc: 0.2440 - classifier_body_acc_1: 0.2489 - classifier_body_acc_2: 0.2480 - classifier_body_acc_3: 0.2460 - classifier_body_acc_4: 0.2431 - val_loss: 31.7953 - val_classifier_body_loss: 6.6873 - val_classifier_body_acc: 0.1130 - val_classifier_body_acc_1: 0.1155 - val_classifier_body_acc_2: 0.1193 - val_cl

KeyboardInterrupt: 

# Save weights

In [ ]:
# save shared_lstm
save_lstm(LSTM_cell, time_str, log_dir)

# save classifier
classifier.save_weights(log_dir + "/classifier_%s.h5" % time_str)

In [ ]:
time_str